In [1]:
import warnings
warnings.filterwarnings("ignore")
import scanpy as sc
import cupy as cp
from cupy.sparse import cupyx as cpx
import cudf
import pandas as pd
import numpy as np
import seaborn as sns
import anndata
from collections import defaultdict
import glob
samples=list(map(lambda p: "_".join(p.split("_")[:2]),glob.glob("*mtx*") ))

In [2]:
samples=glob.glob("**/**/**/filtered*/")

In [3]:
sample=samples[0]
def mtx_to_pandas(sample):
    mtx_data=cudf.read_csv(sample+"matrix.mtx.gz",sep=" ",dtype=['float32' for i in range(3)],comment="%",header=None)
    data=mtx_data.iloc[1:,2].values
    row=mtx_data.iloc[1:,1].values
    col=mtx_data.iloc[1:,0].values

    barcodes=cudf.read_csv(sample+"barcodes.tsv.gz",header=None)["0"]

    genes=cudf.read_csv(sample+"features.tsv.gz",header=None,sep="\t")["1"]

    sp=cpx.scipy.sparse.coo_matrix((data, (row, col)),shape=(len(barcodes)+1,len(genes)+1))
    sp=sp.tocsr()[1:,1:]

    sparse_df=pd.DataFrame.sparse.from_spmatrix(sp.get().astype(int),index=barcodes.to_array(),columns=genes.to_array())

    name_counts = defaultdict(int)
    new_col_names = []
    for name in sparse_df.columns:
        new_count = name_counts[name] + 1
        new_col_names.append("{}{}".format(name, new_count))
        name_counts[name] = new_count 
    sparse_df.columns = new_col_names
    ### appending prefix to cell names according to samples if needed
    sparse_df.index=sample.split("/")[1]+"_"+sparse_df.index
    ### -
    return sparse_df.loc[:,np.random.choice(sparse_df.columns,5000,replace=False)]



In [4]:
from joblib import delayed, Parallel
allsparse_dfs = Parallel(n_jobs=len(samples))(
        delayed(mtx_to_pandas)(
            samples[i]
        )
        for i in range(len(samples))
)
    

In [5]:
allgenes=[sparse_df.columns for sparse_df in allsparse_dfs]
allgenes=np.unique(np.concatenate(allgenes))

def add_missing_genes(sparse_df):
    sparse_df=sparse_df.reindex(columns=allgenes, fill_value=0)
    return sparse_df.astype(pd.SparseDtype("int")).sparse.to_coo()

In [6]:
allsps = Parallel(n_jobs=len(samples))(
        delayed(to_csr_gpu)(
            allsparse_dfs[i]
        )
        for i in range(len(allsparse_dfs))
)

In [7]:
import scipy.sparse
megasparse=scipy.sparse.vstack(allsps).tocsr()

In [8]:
megasparse

<212086x33131 sparse matrix of type '<class 'numpy.int64'>'
	with 52895803 stored elements in Compressed Sparse Row format>